In [1]:
!pip install bert-for-tf2
!pip install transformers

     |████████████████████████████████| 1.3MB 3.4MB/s 
     |████████████████████████████████| 1.1MB 21.0MB/s 
     |████████████████████████████████| 890kB 30.1MB/s 
     |████████████████████████████████| 2.9MB 44.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d4734dff2ac3e01508fb1906e3cdb1184c85ca6aca4031f978b30b8025823fd5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget https://raw.githubusercontent.com/MowlanicaBilla/OpenKP/master/evaluate.py

--2020-10-30 06:03:58--  https://raw.githubusercontent.com/MowlanicaBilla/OpenKP/master/evaluate.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4010 (3.9K) [text/plain]
Saving to: ‘evaluate.py.1’

evaluate.py.1       100%[===================>]   3.92K  --.-KB/s    in 0s      

2020-10-30 06:03:58 (66.7 MB/s) - ‘evaluate.py.1’ saved [4010/4010]



In [3]:
import os
import json
import math
import datetime

from tqdm import tqdm

import pandas as pd
import numpy as np

from tensorflow import keras

import bert # Importing BERT
#from bert import BertModelLayer
#from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
#from bert.tokenization.bert_tokenization import FullTokenizer

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
stopwords = stopwords.words('english')

import evaluate
import sys
import os

import string
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    l = []
    for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
        l.append(subtree.leaves())
    
    return l

def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    #word = stemmer.stem(word)
    #word = lemmatizer.lemmatize(word)
    return word

def acceptable_word(word):
    """Checks conditions for acceptable word: length, stopword."""
    accepted = bool(2 <= len(word) <= 40
        and word.lower() not in stopwords)
    return accepted


def get_terms(tree):
    kp = []
    for leaf in leaves(tree):
        term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
        if term:
            kp.append(term)
    
    return kp

In [5]:
def get_kp(text):
    
    # Used when tokenizing words
    sentence_re = r'''(?x)      # set flag to allow verbose regexps
            (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
          | \w+(?:-\w+)*        # words with optional internal hyphens
          | \$?\d+(?:\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
          | \.\.\.              # ellipsis
          | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
        '''

    #lemmatizer = nltk.WordNetLemmatizer()
    #stemmer = nltk.stem.porter.PorterStemmer()

    #Taken from Su Nam Kim Paper
    grammar = r"""
        NBAR:
            {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns

        NP:
            {<NBAR>}
            {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
    """

    toks = nltk.regexp_tokenize(text, sentence_re)
    #tokens = nltl.tok
    postoks = nltk.tag.pos_tag(toks)
    chunker = nltk.RegexpParser(grammar)
    tree = chunker.parse(postoks)
    terms = get_terms(tree)
    #print(terms)
    pos,pos_set = find_positions(text,terms)
    return terms,pos,pos_set

In [6]:
def get_kp(text):
    
    # Used when tokenizing words
    sentence_re = r'''(?x)      # set flag to allow verbose regexps
            (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
          | \w+(?:-\w+)*        # words with optional internal hyphens
          | \$?\d+(?:\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
          | \.\.\.              # ellipsis
          | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
        '''

    #lemmatizer = nltk.WordNetLemmatizer()
    #stemmer = nltk.stem.porter.PorterStemmer()

    #Taken from Su Nam Kim Paper
    grammar = r"""
        NBAR:
            {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns

        NP:
            {<NBAR>}
            {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
    """

    toks = nltk.regexp_tokenize(text, sentence_re)
    #tokens = nltl.tok
    postoks = nltk.tag.pos_tag(toks)
    chunker = nltk.RegexpParser(grammar)
    tree = chunker.parse(postoks)
    terms = get_terms(tree)
    #print(terms)
    pos,pos_set = find_positions(text,terms)
    return terms,pos,pos_set

In [7]:
def find_positions(document, kps):
  '''
  merge the same kps & keep present kps in document
  Inputs:
  document : a word list : ['sun', 'sunshine', ...] || lower cased
  kps : can have more than one kp : [['sun'], ['key','phrase'], ['sunshine']] || not duplicate
  Outputs:
  all_present_kps : present keyphrases
  positions_for_all : start_end_posisiton for prensent keyphrases
  a present kp postions list : every present's positions in documents,
  each kp can be presented in several postions .
  [[[0,0],[20,21]], [[1,1]]]
  '''
  #document = ' '.join([str(elem) for elem in document.split()])
  document = "".join(document)
  document = ' '.join(e.lower() for e in document.split(" ") if e.isalpha())
  #document = document.lower()
  #print(document)
  document = document.split(" ")
  tot_doc_char = ' '.join(document)
  #print("tot_doc_char:",tot_doc_char)
  positions_for_all = []
  position_start,position_end =[],[]
  all_present_kps = []
  positions_for_all = []
  position_start,position_end =[],[]
  all_present_kps = []
  for kp in kps:
    ans_string = ' '.join(kp)
    #print("Ans string:", ans_string)
    if ans_string not in tot_doc_char:
      continue
    else:
        positions_for_each = []
        # find all positions for each kp
        for i in range(0, len(document) - len(kp) + 1):
          Flag = False
          if kp == document[i:i+len(kp)]:
            Flag = True

          if Flag:
            assert len(kp) >= 1
            positions_for_each.append((i+1, i+len(kp)))
            position_start.append(i+1)
            position_end.append(i+len(kp))


    if len(positions_for_each) > 0 :
      positions_for_all.extend(positions_for_each)
      all_present_kps.append(kp)

  assert len(positions_for_all) >= len(all_present_kps)
  print(all_present_kps)
  if len(all_present_kps) == 0:
    #print("abcd")
    return None
  return [position_start,position_end],set(positions_for_all)

In [8]:
!unzip /content/Inspec.zip

Archive:  /content/Inspec.zip
   creating: Inspec/
   creating: Inspec/docsutf8/
  inflating: Inspec/docsutf8/114.txt  
  inflating: Inspec/docsutf8/1267.txt  
  inflating: Inspec/docsutf8/1792.txt  
  inflating: Inspec/docsutf8/298.txt  
  inflating: Inspec/docsutf8/1074.txt  
  inflating: Inspec/docsutf8/1827.txt  
  inflating: Inspec/docsutf8/1276.txt  
  inflating: Inspec/docsutf8/1565.txt  
  inflating: Inspec/docsutf8/1221.txt  
  inflating: Inspec/docsutf8/58.txt  
  inflating: Inspec/docsutf8/1144.txt  
  inflating: Inspec/docsutf8/2027.txt  
  inflating: Inspec/docsutf8/1984.txt  
  inflating: Inspec/docsutf8/63.txt  
  inflating: Inspec/docsutf8/390.txt  
  inflating: Inspec/docsutf8/1373.txt  
  inflating: Inspec/docsutf8/1536.txt  
  inflating: Inspec/docsutf8/634.txt  
  inflating: Inspec/docsutf8/1889.txt  
  inflating: Inspec/docsutf8/30.txt  
  inflating: Inspec/docsutf8/787.txt  
  inflating: Inspec/docsutf8/915.txt  
  inflating: Inspec/docsutf8/1694.txt  
  inflating

In [9]:
data = 'Inspec/docsutf8/'

In [10]:
keys = 'Inspec/keys/'

In [11]:
files = os.listdir(data)
key_files = os.listdir(keys)

In [12]:
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

def remove_stopwords(text):
    text=[word for word in text if word not in stopword]
    return text

In [13]:

def calculate_f1(y_labels,y_preds,depth,levels):
    precision = []
    recall = []
    f1 = []
    
    for idx,y_label in enumerate(y_labels):
        tp = 0
        p = []
        r = []
        y_label = set(np.where(y_label==1)[0])
        #print(y_preds[idx])
        preds = np.where(y_preds[idx]>0.5)[0]
        for i in range(depth):
            if len(preds)>i:
                if preds[i] in y_label:
                    tp+=1
            p.append(tp/(min(i,len(preds))+1))
            r.append(tp/max(len(y_label),1))
    
    
        level_index = []
        for idx,level in enumerate(levels):
            
            if p[level-1]+r[level-1]>0:
                level_index.append(2/((1/p[level-1])+(2/r[level-1])))
            else:
                level_index.append(0)
        #print('k',level_index)
        
        f1.append(level_index)
    f1 = np.array(f1)
    
    print('F1',np.mean(f1,axis=0))

In [14]:
candidates = []
references = []

for file in files[:10]:
    with open(data+file, 'r') as in_file:

        
        text = in_file.read()
        candidates.append({'url':file,
                            'KeyPhrases':get_kp(text)})
    
    name = file.split('.')[0]
    with open(keys+name+'.key', 'r') as in_file:

        can = in_file.readlines()
        can = [line.rstrip('\n').split() for line in can]
        references.append({'url':file,
                            'KeyPhrases':can})
    


with open('result.json', 'w') as out_file:
    for candidate in candidates:
        json.dump(str(candidate), out_file)
        out_file.write('\n')
with open('keys.json', 'w') as out_file:
    for ref in references:
        json.dump(ref, out_file)
        out_file.write('\n')

[['transactional', 'asynchronous', 'replication', 'scheme'], ['mobile', 'database', 'systems'], ['mobile', 'database', 'systems'], ['mobility'], ['users'], ['significant', 'impact'], ['various', 'replica', 'control'], ['today'], ['tlrsp'], ['model'], ['conflict', 'detection'], ['tlrsp'], ['order'], ['tlrsp', 'model'], ['tlrsp', 'model'], ['efficient', 'support'], ['mobile', 'database', 'systems']]
[['alternative', 'manufacturing', 'control'], ['paper'], ['investigation'], ['effects'], ['dynamic', 'job', 'routing'], ['job'], ['decisions'], ['performance'], ['control', 'system'], ['adaptability'], ['performance'], ['various', 'manufacturing', 'environments'], ['control'], ['paper'], ['various', 'control', 'strategies'], ['benchmark', 'manufacturing', 'system'], ['production', 'system'], ['hybrid', 'strategies'], ['reactive', 'agent', 'mechanisms'], ['allocation', 'strategies'], ['appropriate', 'job'], ['heuristics'], ['performance'], ['job', 'congestion', 'increases']]
[['electronic'], [

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [16]:
sentence_re = r'''(?x)      # set flag to allow verbose regexps
            (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
          | \w+(?:-\w+)*        # words with optional internal hyphens
          | \$?\d+(?:\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
          | \.\.\.              # ellipsis
          | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
        '''

In [17]:
max_len = 0
max_kp = 0
min_len =1e100
all_reps = []
key_positions = []
ref_positions = []
for i,file in enumerate(files[:10]):
    print(i)
    with open(data+file, 'r') as in_file:
        text = in_file.read()
        txt = remove_punctuation(text)
        text_toc = nltk.regexp_tokenize(txt, sentence_re)
        kps,pos,pos_set = get_kp(" ".join(text_toc))
        #print(pos)
        key_positions.append(pos_set)
        #print("Text toc:",text_toc)
        #idx = get_features([" ".join(text_toc)])
        #print([tok in embeddings for tok in text_toc])
        idx = [tok in tokenizer.vocab.keys() for tok in text_toc]
        #idx = [tok in embeddings for tok in text_toc]
        #print(idx)
        #rep = np.zeros((len(text_toc),300),dtype=float)
        rep = np.zeros((len(text_toc),768),dtype=float)
        #print(rep.shape)
        word_list = list(np.array(text_toc)[idx])
        input_ids = tf.constant(tokenizer.encode(word_list))[None, :]
        outputs = model(input_ids)
        last_hidden_states = outputs[0][0][1:-1]
        rep[idx] = last_hidden_states  
        all_reps.append(rep)
        max_len = max(max_len,len(text_toc))
        min_len = min(min_len,len(text_toc))
        max_kp = max(max_kp,len(pos_set))
    
    name = file.split('.')[0]
    print("======")
    with open(keys+name+'.key', 'r') as in_file:

        can = in_file.readlines()
        can = [line.rstrip('\n').split() for line in can]
        ref_pos,ref_set = find_positions(" ".join(text_toc),can)
        ref_positions.append(ref_set)
print(max_len,min_len)

0
[['transactional', 'asynchronous', 'replication', 'scheme'], ['mobile', 'database', 'systems'], ['mobile', 'database', 'systems', 'mobility'], ['users'], ['significant', 'impact'], ['data', 'replication'], ['result'], ['various', 'replica', 'control'], ['today'], ['multidatabase', 'environments'], ['problem'], ['new', 'mobile', 'database', 'replication'], ['transactionlevel', 'resultset', 'propagation', 'tlrsp', 'model'], ['paper'], ['conflict', 'detection'], ['resolution', 'strategy'], ['tlrsp'], ['detail'], ['implementation', 'algorithm'], ['order'], ['performance'], ['tlrsp', 'model'], ['mobile', 'replication', 'schemes'], ['detailed', 'simulation', 'model', 'experimental', 'results'], ['tlrsp', 'model'], ['efficient', 'support'], ['mobile', 'database', 'systems'], ['database', 'consistency']]
[['mobile', 'database'], ['data', 'replication'], ['multidatabase'], ['mobile', 'database', 'replication']]
1
[['alternative', 'manufacturing', 'control'], ['benchmark', 'system'], ['paper']

In [18]:
new_rep = []
for rep in all_reps:
    
    new_rep.append(tf.pad(rep,[[0,max_len-rep.shape[0]],[0,0]]))

In [19]:
masking_layer = tf.keras.layers.Masking()
unmasked_embedding = tf.cast(new_rep, tf.float32)

masked_embedding = masking_layer(unmasked_embedding)

In [20]:
type(key_positions[0])

set

In [21]:
len(key_positions)

10

In [22]:
key_positions1 = [item for sublist in key_positions for item in list(sublist)]

In [23]:
ref_positions1 = [item for sublist in ref_positions for item in list(sublist)]

In [24]:
len(ref_positions1)

84

In [25]:
y_label = []
start_pos = []
end_pos = []
final_positions = []
pos_mask = []
final_kp_list = []
for idx,kp in enumerate(key_positions):
    
    kp = list(kp)
    y_val = [1 if key in ref_positions[idx] else 0 for key in kp]
    y_val.extend([0]*(max_kp-len(kp)))
    start =[key[0]-1 for key in kp]
    end = [key[1]-1 for key in kp]
    final_positions.append(tf.pad([start,end],[[0,0],[0,max_kp-len(start)]]))
    pos_mask.append([True]*len(start)+[False]*(max_kp-len(start)))
    y_label.append(y_val)
    final_kp_list.append(kp)

In [26]:
len(final_positions)

10

In [27]:
len([item for sublist in key_positions for item in list(sublist)])

354

In [28]:
class RNNextractor(tf.keras.layers.Layer):
    
    def __init__(self):
        super(RNNextractor, self).__init__()
        self.bilstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100,
                                                                         return_sequences=True),
                                                     merge_mode=None,
                                                    input_shape=(300,204,))
        self.bilstm2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(5,
                                                                         return_sequences=True),
                                                     merge_mode='ave',
                                                     input_shape=(48,100*4))
        self.dense = tf.keras.layers.Dense(1,activation='sigmoid')
        
    def call(self,inputs):
                                                    
        x = self.bilstm1(inputs[0])
        print(x[0].shape)
        #print(inputs[1])
        mask_start = inputs[1][0]
        #mask_start[mask_start>0] = mask_start[mask_start>0]-1
        mask_end = inputs[1][1]
        #mask_end[mask_end>0] = mask_end[mask_end>0]-1
        print(mask_end.shape)
        start_rep_fr = tf.gather(x[0],mask_start)
        start_rep_bk = tf.gather(x[1],mask_start)
        end_rep_fr = tf.gather(x[0],mask_end)
        end_rep_bk = tf.gather(x[0],mask_end)
        
        print(start_rep_bk.shape)
        span_fe_diff_fr = start_rep_fr-end_rep_fr
        span_fe_prod_fr = tf.math.multiply(start_rep_fr,end_rep_fr)
        span_fe_diff_bk = start_rep_bk-end_rep_bk
        span_fe_prod_bk = tf.math.multiply(start_rep_bk,end_rep_bk)
        print(span_fe_diff_bk.shape)
        span_fe = tf.concat([start_rep_fr,
                             end_rep_fr,
                             start_rep_bk,
                             end_rep_bk,
                             span_fe_diff_fr,
                             span_fe_diff_bk,
                             span_fe_prod_fr,
                             span_fe_prod_bk
                            ],2)
        
        print(span_fe.shape)
        x = self.bilstm2(span_fe)
        x =  self.dense(x)
        return x

In [29]:
extractor = RNNextractor()

In [30]:
f = tf.random.normal([10,5])
pos = tf.constant([[1,5,8],[2,6,9]])

In [31]:
x_train = tf.transpose(new_rep,perm=[0,1,2])
x_pos = tf.stack(final_positions)
#x_pos = tf.tile(tf.expand_dims(x_pos,-1),[1,1,1,100])
y_train = tf.stack(y_label)
y_train = tf.cast(y_train,dtype=float)

In [32]:
final_positions

[<tf.Tensor: shape=(2, 73), dtype=int32, numpy=
 array([[ 15,  21,  74, 114,  55,  81,  58,  10,  93,   1,  49,  27,  58,
          99,  65,  38,  78,  10,  47,  25,  67, 131,  33,  18, 114, 118,
           6, 122,  90,  86,  93, 106,  70,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0],
        [ 15,  22,  74, 114,  59,  82,  59,  13,  94,   4,  52,  29,  58,
         101,  65,  39,  78,  12,  47,  25,  68, 132,  33,  19, 115, 119,
           8, 124,  90,  86,  93, 110,  71,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)>,
 <tf.Tensor: shape=(2, 73), dtype=int32, numpy=
 array([[143,  31,  10, 146,  50,  68,  19, 139,

In [33]:
tf.stack(final_positions).shape

TensorShape([10, 2, 73])

In [34]:
x_train.shape

TensorShape([10, 211, 768])

In [35]:
x_pos.shape

TensorShape([10, 2, 73])

In [36]:
x_pos1 = tf.tile(tf.expand_dims(x_pos,-1),[1,1,1,100])

In [37]:
x_pos1.shape

TensorShape([10, 2, 73, 100])

In [38]:
y_train.shape

TensorShape([10, 73])

In [39]:
res = extractor([x_train[:,0:1,:],x_pos1[:,:,0,0]])
res.shape


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(10, 1, 100)
(2,)


InvalidArgumentError: ignored

In [ ]:
tf.gather(res[:,0,0]-y_train[0],np.where(pos_mask[0])[0])

In [ ]:
ip1 = tf.keras.layers.Input(shape=(260,300))
mask = tf.keras.layers.Masking(mask_value=0.0)(ip1)
#custom_mask = tf.keras.layers.Input(shape=(100,))
bilstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100,
                                                      return_sequences=True),
                                                     merge_mode=None)(mask)
ip2 = tf.keras.layers.Input(shape=(2,77),dtype='int32')
mask_start = ip2[0][0]
mask_end = ip2[0][1]

start_rep_fr = tf.gather(bilstm1[0],mask_start,axis=1)
start_rep_bk = tf.gather(bilstm1[1],mask_start,axis=1)
end_rep_fr = tf.gather(bilstm1[0],mask_end,axis=1)
end_rep_bk = tf.gather(bilstm1[0],mask_end,axis=1)


span_fe_diff_fr = start_rep_fr-end_rep_fr
span_fe_prod_fr = tf.math.multiply(start_rep_fr,end_rep_fr)
span_fe_diff_bk = start_rep_bk-end_rep_bk
span_fe_prod_bk = tf.math.multiply(start_rep_bk,end_rep_bk)


span_fe = tf.keras.layers.concatenate([start_rep_fr,
                     end_rep_fr,
                     start_rep_bk,
                     end_rep_bk,
                     span_fe_diff_fr,
                     span_fe_diff_bk,
                     span_fe_prod_fr,
                     span_fe_prod_bk
                    ],2)
bilstm2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(5,return_sequences=True),
                                         merge_mode='ave',
                                         input_shape=(48,100*4))(span_fe)
output = tf.keras.layers.Dense(1,activation='sigmoid',input_shape=(5,))(bilstm2)

In [ ]:
kpe_model = tf.keras.models.Model(inputs=[ip1,ip2], outputs=output)


In [ ]:
print(kpe_model.summary())

In [ ]:
opt = tf.keras.optimizers.Adamax(learning_rate=0.1)
kpe_model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
pos_weight = tf.cast(pos_mask,dtype='int32')

In [ ]:
kpe_model.fit([x_train[:90],x_pos[:90]], y_train[:90], 
              batch_size=24,epochs=10,
              use_multiprocessing=True,validation_split=0.2)

In [ ]:

#results = kpe_model.evaluate([x_train[90:],x_pos[90:]], y_train[90:], batch_size=8)
#print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = kpe_model.predict([x_train[90:],x_pos[90:]])
print("predictions shape:", predictions.shape)

In [ ]:
calculate_f1(y_train[90:],predictions,30,[5,10])